In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

In [2]:
'''
 Since we are dealing with logistic regression,
 the hypothesis is defined as:

                    1
       F(x) = --------------
                1 + exp^(-x)

 However, its implementation may result in overflow
 if x is too large, then, the version implemented 
 here is more stable with similar results.
'''
def hypothesis(theta,X):
    cX = np.copy(X)
    '''
                    1
       F(x) = --------------
                1 + exp^(-x)
    '''    
    cX[X > 0] = -cX[X > 0]
    h = 1 / (1 + np.exp(np.dot(cX,theta)))

    return h

# Apply a multi class classification of the samples
# regarding an optimized theta
# ToDo
def classify_multiclass(theta , X , th, nmodels):
    # inserting the X^0 coeficient
    X = np.insert(X, 0, 1, axis=1)
    
    # Given a theta collection calculate the hyphotesis
    y = hypothesis(theta, X)
    y[y >= th] = 1
    y[y < th] = 0
    X = np.delete(X, 0, axis=1)
    return y

# Given a threshold apply a 
# binary classification of the samples
# regarding an optimized theta
def classify(theta, X, th):
    X = np.insert(X, 0, 1, axis=1)
    y = hypothesis(theta, X)
    y[y >= th] = 1
    y[y < th] = 0
    X = np.delete(X, 0, axis=1)
    return y

def predict(theta,X):
    X = np.insert(X, 0, 1, axis=1)
    y = hypothesis(theta, X)
    X = np.delete(X, 0, axis=1)
    return y



In [3]:
#-----------------------------------
#   Evaluation Metrics and Loss Functions
#-----------------------------------

def cross_entropy_loss(h, y):
    # y.log(h) + (1-log(h) . 1-y)
    # log probability * inverse of the log probabality 
    return np.multiply(np.log(h),y) + np.multiply((1-np.log(h)),(1-y))

def cost(theta, X, y):
    h = hypothesis(theta, X)
    cost = cross_entropy_loss(h, y)
    mean_cost = cost.sum() / -y.shape[0]
    return mean_cost

def accuracy_score(predY, Y):
    TP = ((predY == Y) & (predY == 1.)).sum()
    TN = ((predY == Y) & (predY == 0.)).sum()
    acc = (TP + TN) / predY.shape[0]
    return acc

def precision_score(predY,Y):
    TP = ((predY == Y) & (predY == 1)).sum()
    FP = ((predY != Y) & (predY == 1)).sum()
    precision = TP / (TP + FP)
    return precision

def recall_score(predY, Y):
    TP = ((predY == Y) & (predY == 1)).sum()
    FN = ((predY != Y) & (predY == 0)).sum()
    recall = TP / (TP + FN)
    return recall

def f1_score(predY, Y, beta):
    precision = precision_score(predY, Y)
    recall = recall_score(predY, Y)
    fscore = (1 + beta * beta) * ((precision * recall) / ((beta * beta * precision) + recall))
    return fscore

In [15]:
#-----------------------------------
#   Gradient Descent
#-----------------------------------

def BGD(X, y, alpha, iterations):

    X = np.insert(X, 0, 1, axis=1)

    nsamples = X.shape[0]
    nfeatures = X.shape[1]
    theta = np.zeros(nfeatures)
    J=[]

    for i in range(iterations):

        h = hypothesis(theta,X)

        error = h - y

        grad = np.dot(X.transpose(),error)/nsamples

        theta = theta - alpha * grad

        J.append(cost(theta,X,y))
#         J.append(error)

    X = np.delete(X,0,axis=1)
    plt.figure()
    plt.plot(J)    
    plt.ylabel('Error')
    plt.xlabel('iterations')
    plt.show()

    return theta, J[iterations-1]

In [ ]:
#-----------------------------------
#   Logistic Regression - Toy Example
#-----------------------------------

size=5000
size2=size//2
size10=size//10
size102=size10//2

X = np.random.randint(low=1, high=99, size=(size,1))
X = np.concatenate((X, X, X), axis=1)
X = np.multiply(X,np.concatenate((np.full((size2,1),1), np.full((size2,1),100))))

print(X)
y = np.empty((size))
y[X[:,1] <= 99]=0
y[X[:,1] >= 100]=1

print("\n--- X")
print(X)
print("\n--- y")
print(y)

theta,error = BGD(X, y, 0.001, 10000)
print("\n--- Theta")
print(theta)
print("\n--- Error")
print(error)


X_val = np.random.randint(low=1, high=9900, size=(size10,1))
X_val = np.concatenate((X_val, X_val, X_val),axis=1)
y_val = np.empty((size10))
y_val[X_val[:,1] <= 99]=0
y_val[X_val[:,1] >= 100]=1


h = predict(theta,X_val)
print("\n--- Hypothesis")
print(h)
predY = classify(theta,X_val,0.7)
print("\n--- Classification")
print(predY)
print("\n--- Expected Output")
print(y_val)

acc = accuracy_score(predY, y_val)
pre = precision_score(predY, y_val)
recall = recall_score(predY, y_val)
f = f1_score(predY, y_val, 1)

print("\n--- Accuracy")
print(acc)
print("\n--- Precision")
print(pre)
print("\n--- Recall")
print(recall)
print("\n--- F1Score")
print(f)


[[  90   90   90]
 [  92   92   92]
 [  29   29   29]
 ...
 [ 200  200  200]
 [3400 3400 3400]
 [7300 7300 7300]]

--- X
[[  90   90   90]
 [  92   92   92]
 [  29   29   29]
 ...
 [ 200  200  200]
 [3400 3400 3400]
 [7300 7300 7300]]

--- y
[0. 0. 0. ... 1. 1. 1.]


In [ ]:
#-----------------------------------
# MultiClass Classification - Toy Example
#-----------------------------------

# Not ready, still has a bug

size=500
size4=size//4
size10=size//10
size104=size10//4
X = np.random.randint(low=0,high=7999, size=(size,1))
X = np.concatenate((X,X,X),axis=1)
y = np.empty((size),dtype=np.float128)
y[X[:,1] <= 1999]=0
y[((X[:,1] <= 3999) & (X[:,1]>1999))]=1
y[((X[:,1] <= 5999) & (X[:,1]>3999))]=2
y[X[:,1]>5999]=3

classes = np.unique(y)
print(classes)
theta = {}

for c in classes:

	cy = np.copy(y)

	cy[y != c] = 0
	cy[y == c] = 1

	theta[c],acc = BGD(X,cy,0.001,500)

X_val = np.random.randint(low=1,high=7999, size=(size10,1))
X_val = np.concatenate((X_val,X_val,X_val),axis=1)
y_val = np.empty((size10))
y_val[X_val[:,1] <= 1999]=0
y_val[((X_val[:,1] <= 3999) & (X_val[:,1]>1999))]=1
y_val[((X_val[:,1] <= 5999) & (X_val[:,1]>3999))]=2
y_val[X_val[:,1]>5999]=3

print("\n--- Theta")
print(theta)

predY, pred = classify_multiclass(theta,X_val)
print("\n--- Hypothesis")
print(pred)
print("\n--- Classification")
print(predY)
print("\n--- Expected Output")
print(y_val)

acc = accuracy_score(predY, y_val)
pre = precision_score(predY, y_val)
recall = recall_score(predY, y_val)
f = f1_score(predY, y_val, 1)

print("\n--- Accuracy")
print(acc)
print("\n--- Precision")
print(pre)
print("\n--- Recall")
print(recall)
print("\n--- F1Score")
print(f)